In [ ]:
#Classify the mail as spam or not using Naïve Bayes classifier. Hard code it to learn the model.
#AKANKSHA DHAR
#NAIVE BAYES


#importing important libraries
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, RocCurveDisplay
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate
import seaborn as sns
from sklearn.metrics import plot_confusion_matrix
from sklearn import metrics
from sklearn.model_selection import train_test_split

# defining function for naive bayes classifier
class NaiveBayes(object):
    def __init__(self):
        self.priors = None
        self.params = None
        self.unique_labels = None
    
    def fit(self, X, y, alpha=1.0):
        assert ((alpha <= 1.0) and (alpha > 0.0)), "ERROR!"
        self.unique_labels = np.unique(y)
        self.params = np.zeros(shape = (X.shape[1], len(self.unique_labels)))
        self.priors = np.zeros(shape = (len(self.unique_labels),))
        
        for ix,label in enumerate(self.unique_labels):
            mask = (y == label)
            token_counts_in_label = (np.sum(X[mask, :], axis=0) + alpha)
            total_tokens_in_label = np.sum(X[mask, :]) + X.shape[1] * alpha
            self.params[:, ix] = token_counts_in_label / total_tokens_in_label
            self.priors[ix] = np.sum(mask)/len(y)
    #Likelihood probabilities
    def predict_log_likelihood(self, X):
        log_params = np.log(self.params)
        log_likelihoods = np.dot(X, log_params)
        print("Naive bayes")
        print()
        print("1. Likelihood probabilities")
        print()
        print()
        print(log_likelihoods)
        print()
        return log_likelihoods
    #Prediction        
    def predict(self, X):
        log_likelihoods = self.predict_log_likelihood(X)
        index_to_label = np.argmax(log_likelihoods, axis=1)
        pred_y = np.asarray([self.unique_labels[index] for index in index_to_label])

        return pred_y
        
    def load_model(self, load_from_path):
        with open(load_from_path, "r") as f:

            self.unique_labels = np.array([int(label) for label in f.readline().split()])
       
            self.priors = np.array([float(p) for p in f.readline().split()])

        param_cols = [i for i in range(1, len(self.priors)+1)]
        self.params = np.loadtxt(load_from_path, skiprows=2, usecols=param_cols)       
#Loading dataset        
df=pd.read_csv("emails.csv")
#defining x and y
X = df.iloc[:, 1:-1].values
y = df.iloc[:, -1].values
#splitting dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)
mnb_model=NaiveBayes()  #calling naive bayes function
mnb_model.fit(X_train,y_train)  #fitting the model

def report(model):
    preds = model.predict(X_test) #prediction

preds = mnb_model.predict(X_test)

#plotting confusion matrix
nb = MultinomialNB()
nb.fit(X_train,y_train)

plot_confusion_matrix(nb,X_test,y_test)

#Accuracy
accuracy = metrics.accuracy_score(y_test, preds)
print("2. The accuracy of the model is found to be ",accuracy)
print()
print()

#recall and precision
print("3.Precision and recall: ")
print(classification_report(preds,y_test))
print()
print()
print("4. Confusion matrix: ")  
